In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from eICU_preprocessing.reader import eICUReader

## Load the data

In [6]:
DATA_PATH = "D:/Projects/CS598/DLH/DLH-Term-Project/data/"
device = torch.device('cuda')

In [7]:
def load_data(data_path=DATA_PATH):
    datareader = eICUReader
    train_datareader = datareader(data_path + 'train', device=device,
                                       labs_only=False, no_labs=True)
    val_datareader = datareader(data_path + 'val', device=device,
                                     labs_only=False, no_labs=True)
    
    return train_datareader, val_datareader

train_loader, val_loader = load_data()

## Define the model

In [ ]:
n_layers = 9

In [ ]:
class TPC(nn.Module):
    def __init__(self):
        super(TPC, self).__init__()
        self.main_dropout = nn.Dropout(p=0.45)
        self.temp_dropout = nn.Dropout(p=0.05)
        self.layer_modules = nn.ModuleDict()
        temp = nn.Conv1d(in_channels=174,
                         out_channels=1044,
                         kernel_size=4,
                         stride=1,
                         groups=87)
        bn_temp = nn.BatchNorm1d(num_features=1044, momentum=0.1)
        point = nn.Linear(in_features=241, out_features=13)
        bn_point = nn.BatchNorm1d(num_features=13, momentum=0.1)

        self.layer_modules["0"] = nn.ModuleDict({
            'temp': temp,
            'bn_temp': bn_temp,
            'point': point,
            'bn_point': bn_point})

    def forward(self, x):
        x_temp = self.temp_dropout(self.layer_modules["0"]['bn_temp'](self.layer_modules["0"]['temp'](x)))
        x_point = self.main_dropout(self.layer_modules["0"]['bn_point'](self.layer_modules["0"]['point'](x)))
        return x